In [6]:
from uproot_methods import TVector2, TVector3, TLorentzVector

Getting ROOT, Python3, and Jupyter happy together on macOS was hard enough. Just going to grab HEPvector from disk. Don't do this in your code, just use pip to install.

In [2]:
import sys
sys.path.insert(0, '..')

In [4]:
from hepvector import Vector, Vector2D, Vector3D, LorentzVector
import numpy as np

We need a way to compare a hepvector to an uproot vector:

In [36]:
def compare(npvec, rvec):
    rveclist = [rvec.x, rvec.y]
    if hasattr(rvec, 'z'):
        rveclist.append(rvec.z)
    if hasattr(rvec, 't'):
        rveclist.append(rvec.t)
        
    rvecarr = np.array(rveclist, np.double)
    return np.allclose(npvec, rvecarr)

# 2D vector comparison

In [15]:
v = Vector2D(1,2)
v2 = Vector2D(.3, .1)
r = TVector2(1,2)
r2 = TVector2(.3, .1)

In [16]:
assert v.x == r.x
assert v.y == r.y

assert v[0] == r.x
assert v[1] == r.y

In [17]:
assert v.phi == r.phi
assert v.rho == r.rho
assert v.mag == r.mag
assert v.mag2 == r.mag2

In [24]:
assert compare(v.unit, r.unit)
assert np.isclose(v.angle(v2), r.angle(r2))
# r.Norm(r2)
# r.Ort()
# r.Proj(r2)
# r.Rotate(phi)
# v.pt()
# v.pt2()

# 3D vector comparison

In [25]:
v = Vector3D(1,2,3)
v1 = Vector3D(.4,-.1,.9)
r = TVector3(1,2,3)
r1 = TVector3(.4,-.1,.9)

In [26]:
assert v.x == r.x
assert v.y == r.y
assert v.z == r.z

In [27]:
r.cottheta

0.7453559924999299

In [30]:
np.tan(v.theta)

0.74535599249993

In [33]:
assert v.mag == r.mag       # get magnitude (=rho=Sqrt(x*x+y*y+z*z)))
assert v.mag2 == r.mag2     # get magnitude squared
assert v.theta == r.theta   # get polar angle
#assert np.cos(v.theta) == r.CosTheta() # get cos of theta
assert np.isclose(np.tan(v.theta), r.cottheta)
assert v.phi == r.phi       # get azimuth angle
#assert v.pt == r.Perp()       # get transverse component
#assert v.pt2 == r.Perp2()     # get transverse component**2

In [12]:
# v.Perp(v1);
# v.Perp2(v1);

In [37]:
# v.PseudoRapidity();
# Patch for bug in uproot_methods:
TVector3.TVector3 = TVector3

assert compare(v.cross(v1), r.cross(r1))
assert v.dot(v1) == r.dot(r1)
assert compare(v + v1, r + r1)
assert compare(v - v1, r - r1)

# r.Rotate* are oddly only inplace
# v.rotate_euler(phi=30)

# Lorentz vector comparison

In [38]:
v = LorentzVector(1,2,3,.2)
v1 = LorentzVector(.4,.1,-.7,2)
r = TLorentzVector(1,2,3,.2)
r1 = TLorentzVector(.4,.1,-.7,2)

In [39]:
assert v.x == r.x
assert v.y == r.y
assert v.z == r.z
assert v.t == r.t
assert v.e == r.t

In [47]:
assert v.beta == r.beta
assert np.isclose(v.dot(v1), r.dot(r1))
assert compare(v + v1, r + r1)
assert compare(v - v1, r - r1)
#assert v.pt == r.Perp()
#assert v.pt2 == r.Perp2()
assert compare(v.boostp3, r.boostp3)
assert v.delta_r(v1) ==  r.delta_r(r1)
assert v.eta == r.eta
assert v1.rapidity == r1.rapidity

# Defined methods

Let's look at all methods available (classmethods too):

In [48]:
import inspect
from itertools import zip_longest

ignore = {x for x,y in inspect.getmembers(np.ndarray)}
ignore |= {'__module__', '_repr_html_', '__slots__'}

insp = lambda cls: {x for x,y in inspect.getmembers(cls)} - ignore

v0 = insp(Vector)
v2 = insp(Vector2D) - v0
v3 = insp(Vector3D) - v2
l4 = insp(LorentzVector) - v3

mc = lambda x: max(max(map(len, x)), 8)

a,b,c,d = 'Vector Vector2D Vector3D LorentzVector'.split()
print(f'{a:{mc(v0)}} | {b:{mc(v2)}} | {c:{mc(v3)}} | {d:{mc(l4)}}')
print(f'{"":-^{mc(v0)}}-|-{"":-^{mc(v2)}}-|-{"":-^{mc(v3)}}-|-{"":-^{mc(l4)}}')
for a,b,c,d in zip_longest(v0, v2, v3, l4, fillvalue=''):
    print(f'{a:{mc(v0)}} | {b:{mc(v2)}} | {c:{mc(v3)}} | {d:{mc(l4)}}')

Vector      | Vector2D | Vector3D                | LorentzVector    
------------|----------|-------------------------|------------------
angle       | Y        | from_cylindrical_coords | from_pt_eta_phi_m
origin      | y        | cross                   | x                
mag         | x        | in_basis                | pseudorapidity   
from_vector | rho      | r                       | phi              
dims        | pt2      | Z                       | t                
unit        | X        | rotate_euler            | NAMES            
mag2        | phi      | z                       | Y                
to_pd       | NAMES    | angle                   | delta_r          
from_pandas | pt       | theta                   | METRIC           
            |          | from_pandas             | e                
            |          | origin                  | from_pt_eta_phi  
            |          | mag                     | pt2              
            |          | from_vect